In [1]:
import numpy as np
import hp2np 
import hexalate
import decam2hp
import jsonMaker
from os import getenv
import pandas as pd
import sys
import  matplotlib.pyplot as plt
import mags

In [2]:
def sort_zone(inner_ra, outer_ra, inner_dec, outer_dec, inner_prob, outer_prob, radThreshold=3., separate=False):
    
    if len(inner_ra) == 0:
        inner_df = pd.DataFrame()
    else:
        inner_df = sort_nearest_highest(inner_ra, inner_dec, inner_prob, radThreshold=radThreshold)
    if len(outer_ra) == 0:
        outer_df = pd.DataFrame()
    else:
        outer_df = sort_nearest_highest(outer_ra, outer_dec, outer_prob, startorder=len(inner_df), radThreshold=radThreshold)
    if separate:
        outer_df['prob_flag'] = ['outer'] * len(outer_df)
        inner_df['prob_flag'] = ['inner'] * len(inner_df)
    return pd.concat([inner_df, outer_df], axis=0)

In [3]:
def sort_nearest_highest(ra, dec, prob, startorder=0., radThreshold=3.):
    # Sort by nearest neighbor, find all neighbors within some radius (radThreshold), and sort by probability.
    
    order = np.ones(ra.size)*-999
    df = pd.DataFrame({'ra':ra, 'dec':dec, 'probs':prob, 'order':order})

    i = startorder ## order counter
    idxmaxprob = df['probs'].idxmax()
    while any(df['order'] == -999):
        # calculate distance from max prob point, and discretize by threshold radius
        mydist = np.sqrt((df.iloc[idxmaxprob]['ra'] - df['ra'])**2 + (df.iloc[idxmaxprob]['dec'] - df['dec'])**2)
        df['distance'] = mydist // radThreshold

        # sort  by  distance with prob breaking ties
        df.sort_values(['distance', 'probs'], ascending=[True, False])
        
        groupdf = df.groupby(['distance']).get_group(0) # get all rows where quantized distance = 0 
        groupdf = groupdf[groupdf.order == -999] # make sure we are only ordering points that haven't been observed 

        # this is a check to ensure we don't get stuck on banana or lobe
        # eg everything in the closest radius has already been observed, check for groups of points further away
        j = 0
        while groupdf.empty:
            try: # Try b/c disconnected blobs make empty groups
                groupdf = df.groupby(['distance']).get_group(j)
                groupdf = groupdf[groupdf['order'] == -999]
            except(KeyError): pass
            j += 1

        neworder = np.arange(i, i + len(groupdf)) 
        groupdf['order'] = neworder
        i += len(groupdf)

        df.loc[groupdf.index.values ,'order'] = groupdf['order']
        idmaxprob = groupdf['probs'].idxmin() # start next iteration with the last value in group

    df['distance'] = mydist
    return df

In [47]:
inner_ra = np.linspace(200, 340, num=10)
inner_dec = np.linspace(0, 30, num=10)
outer_ra = np.linspace(300, 340, num=10)
outer_dec = np.linspace(0, 30, num=10)
inner_prob = [0.75, 0.85, 0.95, 0.7, 0.8, 0.9, 0.72, 0.74, 0.76, 0.86]
outer_prob = [0.45, 0.35, 0.15, 0.05, 0.1, 0.5, 0.44, 0.55, 0.11, 0.33]

In [36]:
# inner_ra = np.array(inner_ra); inner_dec = np.array(inner_dec); 
# outer_ra = np.array(outer_ra); outer_dec = np.array(outer_dec); 
inner_prob = np.array(inner_prob); outer_prob = np.array(outer_prob); 

In [19]:
df = sort_zone(inner_ra, outer_ra, inner_dec, outer_dec, inner_prob, outer_prob, radThreshold=4., separate = True)

In [20]:
expTime_inner = 90
expTime_outer = 90

In [32]:
expTime = [np.ones(inner_ra.size)*expTime_inner]
flt = 'r'
mjd = 60068

In [52]:
def get_hexinfo(ra, dec, prob, expTime, filt, mjd, best=False, camera="decam"):
    """
    Gets information about given hexes. Based on Jim's simplicity.calc
    code.

    Parameters:
    -----------
    ra: np.vector
        vector of hex ra
    dec: np.vector
        vector of hex dec
    prob: np.vector
        vector of hex summed prob from LIGO map
    expTime: np.vector
        vector of exposure times
    filt: str
        telescope filt.
    mjd: float
        modified julian date.
    best: bool
        True/False if use best or not
    camera: str
        telescope camera name to take in account.
        
    Outputs:
    --------
    hexinfo: pd.dataframe
        dataframe with each row representing a hex. contains ra, dec,
        prob, and rise and set times in MJD (minutes), and is sorted by 
        probability.
    """

    # find the highest prob hex
    best_ix = np.argmax(prob)

    # find night statistics
    night, sunset, sunrise = mags.findNightDuration(mjd, camera)
    night = np.float64(night)*24.
    sunset = np.float64(sunset)
    sunrise = np.float64(sunrise)

    # work every minute from sunset to sunrise
    mjd_list = np.arange(sunset, sunrise+1./1440., 1./1440.)
    
    # calculate the limiting magnitudes
    limit_mag = []
    best_limit_mag = []
    moon_sep = []
    moon_phase = []
#     print(sunset)
    obs = mags.observed(ra,dec,prob, sunset, doMaps=False, verbose=False)
    for mjd in mjd_list :
        obs.resetTime(mjd)
        obs.limitMag(filt,exposure=expTime)
        limit_mag.append(obs.maglim)
        best_limit_mag.append(obs.maglim[best_ix])
        moon_sep.append(obs.moonSep[0]*360./2/np.pi)
        moon_phase.append(obs.moonPhase)
    limit_mag = np.vstack(limit_mag)
    ix = limit_mag < 0; limit_mag[ix] = 0
    
    #copied from Jim's code, left it here in case someone wants moon information
    moon_sep = np.array(moon_sep)
    print(moon_sep)
    moon_phase = np.array(moon_phase)
    # moon phase: where 0 = full, 90 equals half, and  180 = new
    # convert to %full
    moon_phase = ((180.-moon_phase)/180.)*100.
    bins = np.arange(21,25.5,0.5)
    
    set_list = [ ]
    rise_list = [ ]

#     print(limit_mag)
    for i in range(len(limit_mag)):
        for j in range(len(limit_mag[i])):
            if limit_mag[i][j] !=0 and limit_mag[i-1][j] == 0:
                rise_list.append([mjd_list[i], ra[j], dec[j]])
            elif limit_mag[i-1][j] !=0 and limit_mag[i][j] == 0:
                set_list.append([mjd_list[i-1], ra[j], dec[j]])
                #print(f'setting j={j}, i={i}, {ra[j]}, {mjd_list[i]}')
    
    set_times = []
    rise_times = []
    #print(rise_list)
    
    
    for i in range(len(ra)):
        sets =[]
        rises=[]
        
        #find rise and set times corresponding to each ra and dec
        for s in set_list:
            #print(s[1])
            if s[1]==ra[i] and s[2]==dec[i]:
                #print(s[0], s[1], ra[i], dec[i])
                sets.append(s[0])
                #print(s[1], s[2], ra[i], dec[i], s[0])
        #find rise time for each ra and dec
        for r in rise_list:
            if r[1]==ra[i] and r[2]==dec[i]:
#                print(f'For {ra[i]}, have {r[0]}')
                rises.append(r[0])

        if len(sets) != 0:
            set_times.append(np.max(sets))
        elif len(sets) == 0:
            set_times.append('N/A')
        if len(rises) !=0:
            rise_times.append(np.min(rises))
        elif len(rises) == 0:
            rise_times.append('N/A')

    #print(rise_times, set_times)
    rise_mins = 1440*(rise_times - sunset)
    set_mins = 1440*(set_times - sunset)
    #create the dataframe
    d = {'RA (deg)': ra, 'DEC (deg)': dec, 'PROBABILITY': prob, 'RISES (MIN SINCE SUNSET)': rise_mins, 'SETS (MIN SINCE SUNSET)':set_mins, 'RISES (MJD)': rise_times, 'SETS (MJD)': set_times}
    info = pd.DataFrame.from_dict(d)
    
    #sort by probability 
    hexinfo = info.sort_values(by='PROBABILITY', ascending=False)
#    print(hexinfo)
    
    return hexinfo

In [53]:
get_hexinfo(inner_ra, inner_dec, inner_prob, 90, flt, mjd)

Loading dust map: .//data/plank-ebv-HFI_CompMap_ThermaDustModel.fits
loading inverse stellar density = probability of recognition map
	 ... the sun is up
	 ... the sun is up
[ 6.95489683  6.96036605  6.96583812  6.971313    6.97679063  6.98227095
  6.98775393  6.99323951  6.99872764  7.00421826  7.00971134  7.01520682
  7.02070465  7.02620478  7.03170717  7.03721177  7.04271853  7.0482274
  7.05373833  7.05925129  7.06476622  7.07028308  7.07580182  7.0813224
  7.08684477  7.09236889  7.09789472  7.1034222   7.1089513   7.11448198
  7.12001418  7.12554788  7.13108302  7.13661956  7.14215746  7.14769669
  7.1532372   7.15877894  7.16432189  7.16986599  7.17541122  7.18095752
  7.18650487  7.19205322  7.19760253  7.20315277  7.20870389  7.21425587
  7.21980866  7.22536223  7.23091654  7.23647156  7.24202724  7.24758355
  7.25314047  7.25869794  7.26425595  7.26981445  7.2753734   7.28093279
  7.28649256  7.2920527   7.29761317  7.30317392  7.30873495  7.3142962
  7.31985765  7.32541927  

,RA (deg),DEC (deg),PROBABILITY,RISES (MIN SINCE SUNSET),SETS (MIN SINCE SUNSET),RISES (MJD),SETS (MJD)
2,231.111111,6.666667,0.95,154.000001,625.000002,60068.059028,60068.386111
5,277.777778,16.666667,0.90,375.000001,699.000002,60068.212500,60068.437500
9,340.000000,30.000000,0.86,696.000002,699.000002,60068.435417,60068.437500
1,215.555556,3.333333,0.85,82.000000,572.000002,60068.009028,60068.349306
4,262.222222,13.333333,0.80,300.000001,699.000002,60068.160417,60068.437500
8,324.444444,26.666667,0.76,610.000002,699.000002,60068.375694,60068.437500
0,200.000000,0.000000,0.75,12.000000,519.000002,60067.960417,60068.312500
7,308.888889,23.333333,0.74,529.000002,699.000002,60068.319444,60068.437500
6,293.333333,20.000000,0.72,451.000001,699.000002,60068.265278,60068.437500
3,246.666667,10.000000,0.70,226.000001,676.000002,60068.109028,60068.421528


In [ ]:
class hex_object(probability, rise_time, set_time, ra, dec):
    self.probability = probability
    self.ra = ra
    self.dec = dec
    
    def rise_and_set:
        
    self.rise_time = rise_time
    self.set_time = set_time
#    def airmass_factor(t, x):
        #get the airmass here based on position
        
#    def get_hex_distance(?):
        #find distance to previous hex
        
#    def slew_time(distance):
        #we probably don’t want to move more than 50% of an exposure. should 
        #be linear with distance. that’s physical time that we will not be 
        #taking pictures. 
    
#    def time_to_set(?):
        #dkjdkjd
        
#    def awesomeness_factor():
        #multiply other factors to get this
        
    